In [2]:
import numpy as np
import pandas as pd

import sys
import os
import json
import logging
from pathlib import Path, PurePath
from collections import OrderedDict
from itertools import chain

import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs

from sklearn.metrics import accuracy_score, top_k_accuracy_score
from scipy.spatial.distance import cdist

In [3]:
val = "cellpainting-test-phenotype-imgpermol.csv"
classes = "cellpainting-split-test-imgpermol.csv"

In [4]:
classes_df = pd.read_csv(classes)
classes_df.set_index("SAMPLE_KEY", inplace=True)
classes_df.index
class_index = classes_df.index.to_numpy()

In [6]:
val_df = pd.read_csv(val)
val_df.set_index("SAMPLE_KEY", inplace=True)
val_df.index
val_index = val_df.index.to_numpy()

In [7]:
cellprofiler_path = "/publicdata/cellpainting/cellprofiler-features-image-level.npz"
cellprofiler_npz = np.load(cellprofiler_path)
rownames = cellprofiler_npz["rownames"]
colnames = cellprofiler_npz["colnames"]
cellprofiler_features = cellprofiler_npz["X"]

In [8]:
#check indexes of classes in full cellprofiler features file
class_indices = np.where(np.in1d(rownames, class_index))[0]

In [9]:
#check that there are no repeated samples
class_ids = rownames[class_indices]
print(len(class_ids))
print(len(set(class_ids)))

2116
2115


In [10]:
class_features = cellprofiler_features[class_indices]

In [11]:
class_ids_unique, class_uq_indices = np.unique(class_ids, return_index=True)
class_features_final = class_features[class_uq_indices]
class_ids_final = class_ids[class_uq_indices]
class_inchis = classes_df.loc[class_ids_final]["INCHIKEY"]

In [ ]:
val_indices = np.where(np.in1d(rownames, val_index))[0]

In [ ]:
# some samples from validation set are repeated in the cellprofiler file, np.unique used because of this
val_features = cellprofiler_features[val_indices]
val_ids = rownames[val_indices]
val_ids_unique, indices = np.unique(val_ids, return_index=True)
val_features_final = val_features[indices]
val_ids_final = val_ids[indices]
val_inchis = val_df.loc[val_ids_final]["INCHIKEY"]

In [32]:
print(val_features_final.shape)
print(class_features_final.shape)

torch.Size([44102, 148])
torch.Size([2115, 148])


In [25]:
val_features_final = torch.from_numpy(val_features_final)
class_features_final = torch.from_numpy(class_features_final)

val_features_final_mnorm = val_features_final / val_features_final.norm(dim=-1, keepdim=True)
class_features_mnorm = class_features_final / class_features_final.norm(dim=-1, keepdim=True)

In [26]:
# Assing one label to each molecule
class_dict = {}

for i, inchi in enumerate(class_inchis): 
    class_dict[inchi] = i

In [27]:
# Create ground truth array with molecule labels. NOTE: there are replicates in the dataset
ground_truth = np.zeros(len(val_inchis), dtype=int)

for i, inchi in enumerate(val_inchis): 
    label = class_dict[inchi]
    ground_truth[i] = int(label)

In [28]:
# Calculating accuracies in several ways

# WAY 1
logits = val_features_final_mnorm @ class_features_mnorm.T
acc = accuracy_score(ground_truth, logits.argmax(axis=1)) * 100.0
print(acc)

0.49657611899687093


In [29]:
# WAY 2
N = ground_truth.shape[0]
(np.array(ground_truth) == logits.argmax(axis=1).numpy()).sum() / N * 100

0.49657611899687093

In [30]:
# WAY 3
ranking = torch.argsort(logits, descending=True)
t = torch.tensor(ground_truth, dtype=torch.int16).view(-1,1)

preds = torch.where(ranking == t)[1]
preds = preds.detach().cpu().numpy()

metrics = {}
for k in [1, 5, 10]:
    metrics[f"R@{k}"] = np.mean(preds < k) * 100
    
print(metrics)

{'R@1': 0.49657611899687093, 'R@5': 1.7527549770985442, 'R@10': 2.8932928211872477}


In [ ]:
# WAY 4
probs = (val_features_final_mnorm @ class_features_mnorm.T).softmax(dim=-1)

metrics_skl = {}
for k in [1, 5, 10]:
    metrics_skl[f"R@{k}"] = top_k_accuracy_score(ground_truth, probs, k=k) * 100
    
print(metrics_skl)

In [31]:
from scipy.stats import binomtest

n_samples = val_features_final.shape[0]
print(n_samples)

mdict, cis = {}, {}

for metric, value in metrics.items():
    successes = int(value * n_samples / 100)
    print(successes)
    btest = binomtest(k=successes, n=n_samples)
    mdict[metric] = btest.proportion_estimate * 100
    cis[metric] = btest.proportion_ci(confidence_level=0.95)
    
print(mdict)
print(cis)

44102
219
773
1276
{'R@1': 0.49657611899687093, 'R@5': 1.7527549770985442, 'R@10': 2.8932928211872477}
{'R@1': ConfidenceInterval(low=0.00433114761083082, high=0.005666823567429807), 'R@5': ConfidenceInterval(low=0.016323324330504, high=0.01879587406047005), 'R@10': ConfidenceInterval(low=0.027388291837404755, high=0.030539984816085917)}
